# Improvement

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('simple_input.csv')

In [3]:
df[df["platform_capacity"]>0.95]['BDC'].count()
# For platforms over 95\% capacity, you should add another percentage of cost (or reduce the margins by another percent)

117

In [4]:
df['pc_0.95'] = np.where(df["platform_capacity"]>0.95, 1, 0)
# creat binary variable for platform capacity

In [5]:
df.head()

,BDC,sales,returns,estimated_margins,platform_capacity,total_distribution_cost,profit,pc_0.95
0,1005006,162664.580,718.5400,0.15,0.857131,7843.433142,23115.39103,0
1,1005011,321341.300,2232.5800,0.15,0.857131,10384.414720,46308.64579,0
2,1005182,959710.440,6415.8000,0.15,0.850000,47031.933310,135939.40600,0
3,1015002,1822491.590,43770.2240,0.15,0.857131,188365.420300,238553.39190,0
4,1015006,5849902.955,39287.3564,0.15,0.857131,228709.662200,837285.89040,0


## Innovation/Seasonal product

In [6]:
innovation = pd.read_csv('percent_of_stores_carrying_by_region.csv')
# Overview: A rough estimate of what percentage of stores in each region carries each product in 2018.

In [7]:
sw_inno = innovation[innovation['region']=='Southwest']

In [8]:
sw_inno.head()

,region,description,BDC,strategy,pct_stores_carrying
7299,Southwest,Bulk Tostito 16.0,20050016.0,Region Option,NaN
7300,Southwest,Flush Items,98001001.0,Region Option,0.009259
7301,Southwest,Food/Service Unflav Ruffles,12050011.0,Region Option,NaN
7302,Southwest,LSS Cheddar & SC Ruffles,12010045.0,Region Option,NaN
7303,Southwest,LSS Onion Maui Style Chip,30010017.0,Region Option,NaN


In [9]:
sw_inno['strategy'].unique()
# strategy: whether the product is an option chosen for the region, 
# an innovation, an ethnic product, and in/out (seasonal product), an experimental product, etc.

array(['Region Option', '(blank)', 'Club Only', 'Ethnic', 'Clip Strip',
       'Pallet', 'Shipper', 'In/Out', 'Innovation 2017',
       'Innovation 2016', 'Test Market', 'LIO (National)', 'Caddy',
       'Stick Strip', 'Stick Strips', 'Regional', 'Innovation 2018',
       'Weight Change 2018'], dtype=object)

In [10]:
len(sw_inno['BDC'].unique()) == len(sw_inno)

True

In [11]:
df_inno = pd.merge(df, sw_inno[['BDC','strategy']], how='left', on=['BDC'])

In [12]:
df_inno.isna().sum() # 5 BDC without strategy data

BDC                        0
sales                      0
returns                    0
estimated_margins          0
platform_capacity          0
total_distribution_cost    0
profit                     0
pc_0.95                    0
strategy                   5
dtype: int64

In [13]:
df_inno = df_inno.dropna() # drop BDC with NA

In [15]:
len(df_inno[df_inno["strategy"]=="Innovation 2018"]) 
# 30 BDCs are innovation products of 2018 

30

In [16]:
df_inno['innovation'] = np.where(df_inno["strategy"]=="Innovation 2018", 1, 0)
# creat binary variable for innovation

In [17]:
len(df_inno)

474

## Category

In [18]:
category = pd.read_csv('sales2018_products.csv.gz')
# category_code, category_description, 
# subcategory_code, subcategory_description: internal classification of products by FritoLay.

In [19]:
category['category_description'].unique()

array(['OTHER QUAKER - DRY', 'OTHER', 'SINGLE SERVE', 'TORTILLA CHIPS',
       'POTATO CHIPS', 'CORN CHIPS', 'CHEESE PUFFS', 'SENSIBLE',
       'ALLIED BRANDS', 'BARS', 'CEREALS', 'HOT CEREALS'], dtype=object)

In [20]:
category['subcategory_description'].unique()

array(['GOLDEN GRAIN PASTA', 'OTHER', 'SINGLE SERVE FLEX',
       'SINGLE SERVE NON FLE', 'TOSTITOS', 'SANTITAS', 'RUFFLES',
       'FRITOS', 'LAYS', 'CHEETOS', 'ROLD GOLD', 'SMARTFOOD', 'DORITO',
       'OTHER POTATO CHIPS', "FL STACY'S", 'SUNCHIPS', 'BAKENETS',
       'MUNCHOS', 'FUNYUNS', 'BAKED', 'CHEWY GRANOLA BARS',
       'UNKNOWN SUB CATEGORY', 'CAPTAIN CRUNCH', 'OATS'], dtype=object)

In [21]:
len(category['category_description'].unique()) # 12 categories for all products

12

In [22]:
len(category['subcategory_description'].unique()) # 24 subcategories for all products

24

In [23]:
category = category[['BDC','category_code','subcategory_code']]

In [24]:
category = category.groupby(['BDC']).agg({'category_code':'first','subcategory_code':'first'}).reset_index()

In [25]:
len(category['BDC'].unique()) == len(category)

True

In [26]:
mid = pd.merge(df_inno[['BDC']], category, how='left', on=['BDC'])

In [27]:
mid.isna().sum()

BDC                 0
category_code       0
subcategory_code    0
dtype: int64

In [28]:
len(mid['category_code'].unique()) # only 8 category for input data

8

In [38]:
mid.groupby('category_code')['BDC'].count()

category_code
1      74
3      64
4      13
5      17
6       5
7     133
8     159
24      9
Name: BDC, dtype: int64

In [39]:
mid.groupby('subcategory_code')['BDC'].count()

subcategory_code
1      55
2      16
3       3
5      35
6      22
7       7
9      13
10     17
12      7
13     15
15    115
16     44
17     98
18      2
19      1
20      2
21     13
87      9
Name: BDC, dtype: int64

In [40]:
len(mid['subcategory_code'].unique()) # only 18 subcategory for input data

18

In [41]:
BDC_category = mid['category_code'].unique()
BDC_subcategory = mid['subcategory_code'].unique()

In [42]:
mid = mid.set_index('BDC')

In [43]:
mid.head()

,category_code,subcategory_code
BDC,,
1005006,8,15
1005011,8,15
1005182,8,15
1015002,4,9
1015006,4,9


In [45]:
mid.to_excel('mid-process.xlsx')

In [52]:
I = mid.index

In [53]:
product_category = pd.DataFrame('',index=I,columns=BDC_category)

In [54]:
for i in I:
    for j in BDC_category:
        if mid.loc[i,'category_code']==j:
            product_category.loc[i,j] = 1
        else:
            product_category.loc[i,j] = 0

In [55]:
product_subcategory = pd.DataFrame('',index=I,columns=BDC_subcategory)

for i in I:
    for j in BDC_subcategory:
        if mid.loc[i,'subcategory_code']==j:
            product_subcategory.loc[i,j] = 1
        else:
            product_subcategory.loc[i,j] = 0

In [56]:
product_category.head()

,8,4,1,5,6,7,3,24
BDC,,,,,,,,
1005006,1,0,0,0,0,0,0,0
1005011,1,0,0,0,0,0,0,0
1005182,1,0,0,0,0,0,0,0
1015002,0,1,0,0,0,0,0,0
1015006,0,1,0,0,0,0,0,0


=============================================================================

In [60]:
df_inno = df_inno.drop(columns=['profit'])

In [61]:
df_inno.head()

,BDC,sales,returns,estimated_margins,platform_capacity,total_distribution_cost,pc_0.95,strategy,innovation
0,1005006,162664.580,718.5400,0.15,0.857131,7843.433142,0,Region Option,0
1,1005011,321341.300,2232.5800,0.15,0.857131,10384.414720,0,Region Option,0
2,1005182,959710.440,6415.8000,0.15,0.850000,47031.933310,0,Region Option,0
3,1015002,1822491.590,43770.2240,0.15,0.857131,188365.420300,0,Innovation 2017,0
4,1015006,5849902.955,39287.3564,0.15,0.857131,228709.662200,0,(blank),0


In [62]:
writer = pd.ExcelWriter('new_input.xlsx')

df_inno.to_excel(writer,sheet_name='Profit',index=False)

product_category.to_excel(writer,sheet_name='Category')

product_subcategory.to_excel(writer,sheet_name='Subcategory')

writer.save()